In [ ]:
from drops2 import sensors
from drops2.utils import DropsCredentials
import numpy as np
import pandas as pd
import datetime as dt
import xarray as xr
import logging, json, warnings, os, time
import netrc
from argparse import ArgumentParser
import rioxarray
import rasterio as rio

In [ ]:
time_start
time_end
drops_settings
logging.info("---> Querying drops dataset...")
DropsCredentials.set(drops_settings["DropsAddress"], drops_settings["DropsUser"], drops_settings["DropsPwd"])
sensors_list_P = sensors.get_sensor_list(drops_settings["DropsSensor"],
                                         geo_win=(drops_settings['lon_left'], drops_settings['lat_bottom'], drops_settings['lon_right'], drops_settings['lat_top']),
                                         group=drops_settings["DropsGroup"])
date_from = time_start.strftime("%Y%m%d%H%M")
date_to = time_end.strftime("%Y%m%d%H%M")

df_pluvio = sensors.get_sensor_data(drops_settings["DropsSensor"], sensors_list_P, date_from, date_to, as_pandas=True)
df_pluvio = df_pluvio.dropna('columns', how='all')

df_pluvio_now = pd.DataFrame(index=['name','lon','lat','value'], columns = df_pluvio.columns)

for col in df_pluvio.columns:
    series = df_pluvio[col]
    series[series<0] = np.nan
    series = series.dropna('rows', how='all')

    df_pluvio_now.loc['lat', col] = [i for i in sensors_list_P.list if i.id == col][0].lat
    df_pluvio_now.loc['lon', col] = [i for i in sensors_list_P.list if i.id == col][0].lng
    df_pluvio_now.loc['name', col] = [i for i in sensors_list_P.list if i.id == col][0].name

    if (len(series>0)) and (date_to in series.keys()):
        df_pluvio_now.loc['value',col] = series.resample(drops_settings['frequency'], closed='right', label='right').agg(pd.DataFrame.sum)[date_to]
        df_pluvio_now.loc['value',col] = series.resample(drops_settings['frequency'], closed='right', label='right')
    else:
        df_pluvio_now.loc['value', col] = np.nan

return df_pluvio_now.dropna('columns', how='any')